In [19]:
import pandas as pd
FOLDER = 20
# read json into pandas dataframe
df = pd.read_json(rf"I:\Facultate\Facultate\AEA\bnb_rust\1749144282\experiment_results_{FOLDER}_cfg_1.json")
print(df.shape)
# display the dataframe
print(df['runs'][0][0])

(2, 2)
{'run_number': 0, 'execution_time': 0.9942219257354731, 'solution': [23, 12, 28, 25, 14, 29, 26, 8, 3], 'report': {'file_name': 'bremen_subgraph_20.hgr', 'opt': 9, 'branching_steps': 1, 'upper_bound_improvements': [{'new_bound': 10, 'branching_steps': 0, 'runtime': 0.0003585}, {'new_bound': 9, 'branching_steps': 1, 'runtime': 0.0086268}], 'settings': {'enable_local_search': True, 'enable_max_degree_bound': True, 'enable_sum_degree_bound': True, 'enable_efficiency_bound': True, 'enable_packing_bound': True, 'enable_sum_over_packing_bound': True, 'packing_from_scratch_limit': 3, 'greedy_mode': 'AlwaysBeforeExpensiveReductions', 'time_limit': 600, 'initial_hitting_set': None, 'stop_at': 0}, 'root_bounds': {'max_degree': 5, 'sum_degree': 6, 'efficiency': 7, 'packing': 8, 'sum_over_packing': 8, 'greedy_upper': 10}, 'runtimes': {'total': 0.0088034, 'greedy': 0.0011034999999999999, 'max_degree_bound': 3.1700000000000005e-05, 'sum_degree_bound': 0.0001131, 'efficiency_bound': 0.0001496,

In [ ]:

FOLDER = 250
# read json into pandas dataframe
# 1749150838 >= 200
# 1749144282 <  200
df = pd.read_json(rf"I:\Facultate\Facultate\AEA\bnb_rust\1749150838\experiment_results_{FOLDER}_cfg_1.json")

ts = [r['report']['runtimes']['total'] for r in df['runs'][0]]
slls = [r['report']['opt'] for r in df['runs'][0]]
import numpy as np
print(f"{np.mean(ts):.6f}")
print(f"{np.std(ts):.6f}")
print(f"{np.mean(slls):.6f}")
print(f"{np.std(slls):.6f}")


FileNotFoundError: File I:\Facultate\Facultate\AEA\bnb_rust\1749144282\experiment_results_250_cfg_1.json does not exist